# Ways to Use/Create Functions Dynamically


In [1]:
# from https://www.youtube.com/watch?v=H6HVcU1bF5U&list=WL&index=76&t=630s
from platform import system

class Baz:
    match system():
        case 'Linux':
            def f(self, *args, **kwargs):
                print('linux')
                pass
        case _:
            def f(self, *args, **kwargs):
                print('other')
                pass
b = Baz()
b.f()

linux


## Typing


In [2]:
from typing import Literal

def foo(a, *, opts: Literal['a', 'b', 'c'] = 'a'):
    return opts

foo(None, opts='d')

'd'

## Updating Metadata

Updating `co_consts`

In [3]:
from types import CodeType, FunctionType 
# bytecode

def update(f, **kwargs):
    '''Function that performs a functional update on a function'''

    old = f.__code__
    attrs = [
      'co_argcount', 'co_posonlyargcount', 'co_kwonlyargcount', 'co_nlocals', 'co_stacksize', 'co_flags', 'co_code', 'co_consts', 'co_names', 'co_varnames', 'co_filename', 'co_name', 'co_firstlineno', 'co_lnotab', 'co_freevars', 'co_cellvars' 
    ]

    new = CodeType(*(kwargs.get(a, getattr(old, a)) for a in attrs))

    return FunctionType(
        new, 
        f.__globals__, f.__name__, f.__defaults__, f.__closure__
    )


CodeType.__doc__

'Create a code object.  Not for the faint of heart.'

In [4]:
def addone(a):
    return a + 1


In [5]:
addtwo = update(addone, co_consts=(None, 2))

In [6]:
getattr(addtwo.__code__, 'co_consts', None)

(None, 2)

In [7]:
addtwo(2)

4

## Updating `co_code`

In [8]:
from dis import dis

dis(addone) # 23 means add

  2           0 LOAD_FAST                0 (a)
              2 LOAD_CONST               1 (1)
              4 BINARY_ADD
              6 RETURN_VALUE


In [15]:
def add_to_mul(f):
    old = f.__code__.co_code
    new = old.replace(bytes([23]), bytes([20]))
    return update(f, co_code=new)

my_multwo = add_to_mul(addtwo)

In [16]:
my_multwo(5)

10

In [12]:
codestring = """
import pandas as pd

df = pd.DataFrame({
    'id': ['a']*10
})

"""

exec(codestring)

# Dynamic Tests

In [14]:
# will dynamically generate tests for PyTest

def generate_tests(fn, inputs, outputs):
    for i, o in zip(inputs, outputs):
        exec(f'def test_{inputs.index(i)}():\n    assert fn({i}) == {o}\n')

def largest_odd(lst):
    return max([i for i in lst if i%2!=0])

inputs = [[1,2,3,4], [5,6,7,8], [9,10,11,12]]
outputs = [3,7,11]
generate_tests(largest_odd, inputs, outputs)

In [26]:
from pandas import DataFrame
inputs = {
    'test1': DataFrame({'id': [10]*2}) ,
    'test2': DataFrame({'id': [10]*2}) ,
    'test3': DataFrame({'id': [10]*2}) ,
    'test4': DataFrame({'id': [10]*2}) ,
}

for i in inputs:
    fixture_code = f"""
import pytest
    
@pytest.fixture
def input{i}():
    return inputs[i]

def test{i}(input{i}):
    assert True
    """
    exec(fixture_code)


<module 'pytest' from '/home/aadi/miniconda3/envs/basic_clean/lib/python3.10/site-packages/pytest/__init__.py'>